In [1]:
import pymongo
import dns
import pandas as pd
import io
import datetime

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client.traffic
INTER_TRAVEL_TIME_AVG=5 #average travelling time from gantryFrom to gantryTo in minute
INTERCHAGE_TRAVELTO_GANTRY_TIME_AVG=3  #average travelling time from interchange to 1st gantryTo in minute

In [3]:
def thruGantry(gantryID, begin, timeRange):
    #begin=datetime.datetime.strptime(timeStamp, "%Y/%m/%d %H:%M") #"%Y-%m-%d %H:%M") # Time Format2019-06-24 06:40
    end=begin+ datetime.timedelta(minutes = timeRange) #timeRange in minutes
    coll = db['M03A']
    match={"GantryId":gantryID, "TimeInterval": {"$gte": begin, "$lt": end}}
    groupby = 'VehicleType'
    group = {'_id': "$%s" % (groupby if groupby else None),
             'total': {'$sum': '$交通量'}}   #將各車型分開加總
    result = coll.aggregate([
            { "$match": match },
            { "$group": group}])
    
    df =  pd.DataFrame(list(result))
    return df

In [4]:
def offInterchange(gantryID, begin, timeRange):
    #begin=datetime.datetime.strptime(timeStamp, "%Y/%m/%d %H:%M") #"%Y-%m-%d %H:%M") # Time Format2019-06-24 06:40
    end=begin+ datetime.timedelta(minutes = timeRange) #timeRange in minutes
    coll = db['M08A']
    match={"GantryTo":gantryID, "TimeInterval": {"$gte": begin, "$lt": end}}
    groupby = 'VehicleType'
    group = {'_id': "$%s" % (groupby if groupby else None),
             'total': {'$sum': '$交通量'}}
    result = coll.aggregate([
            { "$match": match },
            { "$group": group}])
    
    df =  pd.DataFrame(list(result))
    return df

In [5]:

def intoInterchange(gantryFrom, gantryTo , timeStamp, timeRange):
    
    fromGntry=thruGantry(gantryFrom, timeStamp-datetime.timedelta(minutes =INTER_TRAVEL_TIME_AVG), timeRange)
    fromGntry.total = fromGntry.total.apply(lambda x:-x) # to transfer add  to minus
    #print('fromGntry',fromGntry)
    toGntry=thruGantry(gantryTo , timeStamp, timeRange)
    #print('toGntry', toGntry)
    diff=toGntry.set_index('_id').add(fromGntry.set_index('_id'), fill_value=0).reset_index()
    #print('diff',diff)
    
    offIntr=offInterchange(gantryFrom,timeStamp,timeRange)
    #print('offIntr',offIntr)
    result=diff.set_index('_id').add(offIntr.set_index('_id'), fill_value=0).reset_index()
    result.sort_values('_id', inplace=True)
    result.rename(columns={'_id':'VehicleType'}, inplace=True)
    print(result)
    return result

In [6]:

def compareInDiff(timeStamp, timeRange):
    begin=datetime.datetime.strptime(timeStamp, "%Y/%m/%d %H:%M") + datetime.timedelta(minutes =INTERCHAGE_TRAVELTO_GANTRY_TIME_AVG) #"%Y-%m-%d %H:%M")
    print('\n', 'After time,',timeStamp)
    dfA=intoInterchange('01F2156N', '01F2089N' , begin, timeRange)
    print('Before time',timeStamp)
    delta=datetime.timedelta(minutes = timeRange)
    dfB=intoInterchange('01F2156N', '01F2089N' , begin-delta, timeRange)
    print('\n\n')

In [7]:
print('The CMS infoed: 204~198K壅塞車速40以下請改道','\n')
compareInDiff("2019/04/05 13:21",60) #2019-04-05T13:21
#compareInDiff("2019/06/26 08:23",60)

print('\n','\n','---The time travel compared---','\n')
compareInDiff("2019/04/04 13:21",60)
compareInDiff("2019/04/06 13:23",60)
compareInDiff("2019/04/07 13:21",60)


The CMS infoed: 204~198K壅塞車速40以下請改道 


 After time, 2019/04/05 13:21
   VehicleType  total
0            5   -1.0
1           31  444.0
2           32   83.0
3           41    5.0
4           42    8.0
Before time 2019/04/05 13:21
   VehicleType  total
0            5   -2.0
1           31  685.0
2           32  202.0
3           41    7.0
4           42   18.0




 
 ---The time travel compared--- 


 After time, 2019/04/04 13:21
   VehicleType  total
0            5      6
1           31    577
2           32    202
3           41      0
4           42     26
Before time 2019/04/04 13:21
   VehicleType  total
0            5    9.0
1           31  345.0
2           32   84.0
3           41    8.0
4           42   14.0




 After time, 2019/04/06 13:23
   VehicleType  total
0            5      4
1           31    688
2           32    143
3           41     20
4           42     10
Before time 2019/04/06 13:23
   VehicleType  total
0            5      9
1           31    783
2           3

In [ ]:
print('The CMS infoed: 204~198K壅塞車速40以下請改道','\n')
compareInDiff("2019/04/05 13:21",60) #2019-04-05T13:21
#compareInDiff("2019/06/26 08:23",60)

print('\n','\n','---The time travel compared---','\n')
compareInDiff("2019/04/04 13:21",60)
compareInDiff("2019/04/06 13:23",60)
compareInDiff("2019/04/07 13:21",60)